In [1]:
import cv2
import numpy as np

img= cv2.imread("test_image.jpg")
np_img = np.copy(img)

def canny(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5),0)
    canny = cv2.Canny(blur, 50, 150)
    return canny

def ROI(img):
    mask = np.zeros_like(img)
    img_height = img.shape[0]
    triangle_points = np.array([[(200, img_height),(1100, img_height),(550, 250)]])
    cv2.fillPoly(mask, triangle_points, 255)
    masked_img = cv2.bitwise_and(img, mask)
    return masked_img


def display_lines(img, lines_coordinates):
    line_img = np.zeros_like(img)
    if lines_coordinates is not None:
        for i in lines_coordinates:
            x1, y1, x2, y2 = i.reshape(-1)
            cv2.line(line_img, (x1,y1), (x2,y2), (255,0,0), 10)
    return line_img

def make_coordinates(img, line_parameters):
    # y = m*x + b
    slope, intercept = line_parameters
    y1 = img.shape[0]
    x1 = int( (y1-intercept)/slope )
    y2 = int(y1*3/5)
    x2 = int( (y2-intercept)/slope )
    return np.array([x1, y1, x2, y2])

def average_slope_intercept(img, lines_coordinates):
    left_fit = []
    right_fit = []
    for i in lines_coordinates:
        x1, y1, x2, y2 = i.reshape(-1)
        parameters = np.polyfit(x = (x1,x2), y = (y1, y2), deg = 1)
        slope = parameters[0]
        intercept = parameters[1]
        if slope <0:
            left_fit.append([slope, intercept])
        else:
            right_fit.append([slope, intercept])
    average_left_fit = np.average(left_fit, axis = 0)
    average_right_fit = np.average(right_fit, axis=0)
    left_fit_coordinates = make_coordinates(img, average_left_fit)
    right_fit_coordinates = make_coordinates(img, average_right_fit)
    return np.array([left_fit_coordinates, right_fit_coordinates])

#canny_img = canny(np_img)
#masked_img = ROI(canny_img)
#lines_coordinates = cv2.HoughLinesP(masked_img, 2, np.pi/180, 100, minLineLength = 40, maxLineGap = 5)
#average_lines_coordinates = average_slope_intercept(np_img, lines_coordinates)
#line_img = display_lines(np_img, average_lines_coordinates)
#combo_img = cv2.addWeighted(np_img, 0.8, line_img, 1, 1)
#cv2.imshow("result", combo_img)
#cv2.waitKey(0)

cap = cv2.VideoCapture("test2.mp4")
while cap.isOpened():
    success, frame = cap.read()
    canny_img = canny(frame)
    masked_img = ROI(canny_img)
    lines_coordinates = cv2.HoughLinesP(masked_img, 2, np.pi/180, 100, minLineLength = 40, maxLineGap = 5)
    average_lines_coordinates = average_slope_intercept(frame, lines_coordinates)
    line_img = display_lines(frame, average_lines_coordinates)
    combo_img = cv2.addWeighted(frame, 0.8, line_img, 1, 1)
    cv2.imshow("result", combo_img)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()